In [1]:
import pandas as pd
import numpy as np
from pythainlp import word_tokenize
from pythainlp.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

ImportError: cannot import name 'stopwords' from 'pythainlp.corpus' (C:\Users\tommy\AppData\Local\pypoetry\Cache\virtualenvs\is-project-thailand-tourism-recommendation-7mwBDOoH-py3.11\Lib\site-packages\pythainlp\corpus\__init__.py)

In [ ]:
# Define Thai stop words
thai_stopwords = set(stopwords.words('thai'))

# Function to normalize text
def normalize_text(text):
    tokens = word_tokenize(text, engine='newmm')
    tokens = [token for token in tokens if token not in thai_stopwords and token.isalpha()]
    return ' '.join(tokens)